In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [79]:
input_text = au.read_txt_file_lines('input.txt')
input_fixed = []
for l in input_text:
    input_fixed.append(l.replace('\\', 'm'))
input_text = input_fixed 
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 110 rows and 110 cols


## plan  part 1 
bit like day 10 .. But bit too different so didnt re-use code but copy pasted some bits. 

**Update:** still a bit tricky, struggled a bit with checking all the conditions. Was working with presence only map but realised in the end that directional map is of course needed to allow multiple passes at mirrors. Complicated things for myself ... 

In [81]:

## print unique characters
print('unique characters:')
print(np.unique(np.array(list(itertools.chain(*input_text)))))

unique characters:
['-' '.' '/' 'm' '|']


In [82]:
start_point = collections.namedtuple('start_point', ['pos', 'dir'])

def traject_pipe(input_text, verbose=0,
                 new_input=None, new_input_char='presence_only',
                 start=collections.namedtuple('start_point', ['pos', 'dir'])((0, 0), 'S')):
    list_new_start_points = []
    n_steps = 0 
    curr_dir = start.dir
    curr_pos = start.pos
    assert new_input_char in ['presence_only', 'presence_and_dir']
    max_steps = 10000 
    if verbose > 0:
        print(f'starting at {curr_pos}')
    continue_traj = True
    while continue_traj:
        n_steps += 1
        curr_row, curr_col = curr_pos
        
        if curr_col >= 0 and curr_row >= 0 and curr_col < len(input_text[0]) and curr_row < len(input_text): 
            pass 
        else:
            if verbose > 0:
                print(f'out of bounds at {curr_pos}')
            continue_traj = False
            break

        ## option to create new map (new_input)
        if new_input is not None:
            if new_input_char == 'presence_only':  # map pipe piece
                new_input[curr_row][curr_col] = '#'
            elif new_input_char == 'presence_and_dir':  # map pipe piece and direction
                new_input[curr_row][curr_col].append(curr_dir)
            else:
                pass

        ## calculate next position
        if curr_dir == 'N':
            next_pos = (curr_row - 1, curr_col)
        elif curr_dir == 'E':
            next_pos = (curr_row, curr_col + 1)
        elif curr_dir == 'S':
            next_pos = (curr_row + 1, curr_col)
        elif curr_dir == 'W':
            next_pos = (curr_row, curr_col - 1)

        if next_pos[1] >= 0 and next_pos[0] >= 0 and next_pos[1] < len(input_text[0]) and next_pos[0] < len(input_text): 
            repeated_traj = False 
        else:
            if verbose > 0:
                print(f'out of bounds at {next_pos}')
            continue_traj = False
            break

        ## find new point:
        next_point = input_text[next_pos[0]][next_pos[1]]

        ## check end:
        if next_point == '.':
            curr_pos = next_pos  # curr_dir stays the same
        elif next_point == '-':
            if curr_dir in ['E', 'W']:
                curr_pos = next_pos  # curr_dir stays the same
            elif curr_dir in ['N', 'S']:
                if verbose > 0:
                    print(f'hit - at {next_pos}')
                list_new_start_points.append(start_point(next_pos, 'E'))
                list_new_start_points.append(start_point(next_pos, 'W'))
                continue_traj = False
                break
        elif next_point == '|':
            if curr_dir in ['N', 'S']:
                curr_pos = next_pos
            elif curr_dir in ['E', 'W']:
                if verbose > 0:
                    print(f'hit | at {next_pos}')
                list_new_start_points.append(start_point(next_pos, 'N'))
                list_new_start_points.append(start_point(next_pos, 'S'))
                continue_traj = False
                break
        elif next_point == '/':
            if curr_dir in ['N', 'S']:
                curr_dir = 'E' if curr_dir == 'N' else 'W'
                curr_pos = next_pos
            elif curr_dir in ['E', 'W']:
                curr_dir = 'N' if curr_dir == 'E' else 'S'
                curr_pos = next_pos
        elif next_point == 'm':  # '\'
            if curr_dir in ['N', 'S']:
                curr_dir = 'W' if curr_dir == 'N' else 'E'
                curr_pos = next_pos
            elif curr_dir in ['E', 'W']:
                curr_dir = 'S' if curr_dir == 'E' else 'N'
                curr_pos = next_pos
        else:
            assert False, f'unknown point {next_point} at {curr_pos}'

        if new_input_char == 'presence_only' and new_input[next_pos[0]][next_pos[1]] == '#':
            repeated_traj = True
        elif new_input_char == 'presence_and_dir' and curr_dir in new_input[next_pos[0]][next_pos[1]]:
            repeated_traj = True 
        if repeated_traj:
            if verbose > 0:
                print(f'hash already exists at {next_pos}')
            continue_traj = False
            break

        if n_steps > max_steps:
            if verbose > 0:
                print('max steps reached')
            continue_traj = False
            break

    return new_input, list_new_start_points, n_steps

In [86]:
def get_n_hash_for_start(start_tile=start_point((0, 0), 'S')):

    list_start_points = [start_tile]
    # new_input =[['.' for _ in range(n_cols)] for __ in range(n_rows)]
    new_input = [[[] for _ in range(n_cols)] for __ in range(n_rows)]
    n_loops = 0
    while len(list_start_points) > 0:
        n_loops += 1
        if n_loops > 10000:
            print('MAX LOOPS REACHED')
            break
        elif n_loops % 1000 == 0:
            print(f'loop {n_loops}, {len(list_start_points)} start points remaining: {list_start_points}')
        new_start_point = list_start_points.pop(0)
        # print(f'new start point: {new_start_point}')
        new_input, list_new_start_points, n_steps = traject_pipe(input_text=input_text, new_input=new_input, verbose=0,
                                                                    new_input_char='presence_and_dir', start=new_start_point)
        list_start_points.extend(list_new_start_points)
        # print(f'new list_start_points: {list_start_points}')

    n_hash = 0
    presence_only_map = copy.deepcopy(new_input)
    for ii in range(len(presence_only_map)):
        for jj in range(len(presence_only_map[0])):
            if len(presence_only_map[ii][jj]) > 0:
                presence_only_map[ii][jj] = '#'
                n_hash += 1
            else:
                presence_only_map[ii][jj] = '.'
    return n_hash

get_n_hash_for_start()

7074

## part 2

Easy with what we got. Actually the code below worked right away, even thought technically it's not correct.. (Problem is that I initiate the edge tile away from the edge, while if that edge tile is a mirror it should change direction. I guess the best solution happened to not be like that. (And I just wanted to try before changing the initial condition))

In [87]:
n_max = 0 

for ii in range(n_rows):
    tmp_n_hash = get_n_hash_for_start(start_tile=start_point((ii, 0), 'E'))
    if tmp_n_hash > n_max:
        n_max = tmp_n_hash
    tmp_n_hash = get_n_hash_for_start(start_tile=start_point((ii, n_cols - 1), 'W'))
    if tmp_n_hash > n_max:
        n_max = tmp_n_hash

for jj in range(n_cols):
    tmp_n_hash = get_n_hash_for_start(start_tile=start_point((0, jj), 'S'))
    if tmp_n_hash > n_max:
        n_max = tmp_n_hash
    tmp_n_hash = get_n_hash_for_start(start_tile=start_point((n_rows - 1, jj), 'N'))
    if tmp_n_hash > n_max:
        n_max = tmp_n_hash

In [88]:
n_max

7530